In [10]:
import serial
import json
import numpy as np

# Prepare data to send
file_path = "C:\\Users\\HOME\\Desktop\\연대 관련\\Year 3 sem 1\\CommsNetwork\\WiFiLocalization\\data\\0_0_0.json"
macs = []
rssi = []
with open(file_path, "r") as f: # read data
    data = json.load(f)
    for d in data:
        d = json.loads(d)
        macs.append(d["MAC"])
        rssi.append(d["RSSI"])

#argsort rssi
rssi = np.array(rssi)
idx = np.argsort(rssi)[::-1] #holds index in descending order, so reverse it
rssi = rssi[idx]
#get top N
rssi = rssi[:5]

#convert to tx power values
tx = []
for i in range(len(rssi)):
    if rssi[i] > -70:
        tx.append(80)
    elif rssi[i] > -90:
        tx.append(60)
    elif rssi[i] > -100:
        tx.append(20)
    else:
        tx.append(8)   

# Open serial port
ser = serial.Serial('COM7', 115200)

# Read data from serial port
while True:
    data = ser.readline()
    data = data.decode("utf-8")

    if "Waiting for input" in data:
        print("connected")
        break

#bake up the command string
cmd = ""
for i in range(len(rssi)):
    j = idx[i]
    #numpy int64 rssi number to string
    target = str(tx[i])
    cmd += macs[j] + ";" + target + ","

#send cmd to serial port
ser.write(cmd.encode("utf-8"))
print("sent")
try:
    while True:
        data = ser.readline().decode("utf-8")
        print(data)
except:
    ser.close()
    print("closed")


connected
sent
Waiting for input

34:8A:12:C9:22:40

80

34:8A:12:C9:22:41

80

34:8A:12:C9:22:43

80

34:8A:12:C9:22:42

80

18:64:72:F4:48:02

60

34:8A:12:C9:22:40

34:8A:12:C9:22:41

34:8A:12:C9:22:43

34:8A:12:C9:22:42

18:64:72:F4:48:02

Setup complete

Setting channel to: 2

Channel set to: 2

Setting power to: 80

Power set to: 78

Setting channel to: 3

Channel set to: 3

Setting power to: 80

Power set to: 78

Setting channel to: 4

Channel set to: 4

Setting power to: 80

Power set to: 78

Setting channel to: 5

Channel set to: 5

Setting power to: 80

Power set to: 78

Setting channel to: 6

Channel set to: 6

Setting power to: 60

Power set to: 60

Attack complete. Time taken:30

Packets sent: 5

Setting channel to: 7

Channel set to: 7

Setting power to: 80

Power set to: 78

Setting channel to: 8

Channel set to: 8

Setting power to: 80

Power set to: 78

Setting channel to: 9

Channel set to: 9

Setting power to: 80

Power set to: 78

Setting channel to: 10

Channel set

## Manually Sending For Debugging

In [14]:
import serial
#Manually Send data to serial port
macs = ['34:8A:12:C9:22:8','34:8A:12:C9:22:24','34:8A:12:C9:22:44','34:8A:12:C9:22:64','34:8A:12:C9:22:84']
rssi = ['8','24','44','64','84']

#open serial port
ser = serial.Serial('COM7', 115200)

#wait for connection
while True:
    data = ser.readline()
    data = data.decode("utf-8")

    if "Waiting for input" in data:
        print("connected")
        break

#bake up the command string
cmd = ""
for i in range(len(rssi)):
    cmd += macs[i] + ";" + rssi[i] + ","

#send cmd to serial port
ser.write(cmd.encode("utf-8"))
print("sent")

try:
    while True:
        data = ser.readline().decode("utf-8")
        print(data)
except:
    ser.close()
    print("closed")

connected
sent
Waiting for input

34:8A:12:C9:22:8

8

34:8A:12:C9:22:24

24

34:8A:12:C9:22:44

44

34:8A:12:C9:22:64

64

34:8A:12:C9:22:84

84

34:8A:12:C9:22:8

34:8A:12:C9:22:24

34:8A:12:C9:22:44

34:8A:12:C9:22:64

34:8A:12:C9:22:84

Setup complete

Setting power to: 8

Power set to: 8

Setting power to: 24

Power set to: 20

Setting power to: 44

Power set to: 44

Setting power to: 64

Power set to: 60

Setting power to: 84

Power set to: 78

Attack complete. Time taken:250

Packets sent: 15

Setting power to: 8

Power set to: 8

Setting power to: 24

Power set to: 20

Setting power to: 44

Power set to: 44

Setting power to: 64

Power set to: 60

Setting power to: 84

Power set to: 78

Attack complete. Time taken:250

Packets sent: 30

Setting power to: 8

Power set to: 8

Setting power to: 24

Power set to: 20

Setting power to: 44

Power set to: 44

Setting power to: 64

Power set to: 60

Setting power to: 84

Power set to: 78

Attack complete. Time taken:250

Packets sent: 

In [11]:
ser = serial.Serial('COM7', 115200)
try:
    while True:
        data = ser.readline().decode("utf-8")
        print(data)
except Exception as e:
    print(e)
    ser.close()
    print("closed")

SerialException: could not open port 'COM7': PermissionError(13, 'Access is denied.', None, 5)